# Title (give a name to your project, e.g., Assignment 1, ML Toolbox-1)

Task/Problem Statement: 
- Briefly describe your project/task/problem

### Dataset
- Provide information about the dataset



##### Description: 

###### Source:
- It could be a URL



##### Input variables:
- Provide a list

##### Output variable: 
- 




In [ ]:
# Import python modules
# For example, import pandas as pd



## Load Data & Displat Dataset Information

In [ ]:
# Single line comments on a code block should be added by using the pound/hash sign shown in this comment

'''
Multi-line comments should be added 
using the syntax shown in this two-line comment
'''


## Data Pre-processing


## Data Analysis

## Create Data Matrix X and Target y

## Partition the Dataset into Train & Test Subsets

## Standardize the Data

## Model Selection

In [ ]:
%%time



## Train a Model Using Optimal Hyperparameters

In [ ]:
%%time

## Evaluate the Model 

In [ ]:
%%time

